In [44]:
#Bring in dependencies
# %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from ast import literal_eval

In [55]:
#Import the data
temp = pd.read_csv('temp.csv')
temp.head()

,Unnamed: 0,title,budget,revenue,release_date,runtime,original_language,production_companies,production_countries,genres,spoken_languages,id,imdb_id,vote_average,vote_count,log rev
0,0,Toy Story,30000000,373554033,10/30/1995,81,en,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'iso_639_1': 'en', 'name': 'English'}]",862,tt0114709,7.7,5415,19.738573
1,1,Jumanji,65000000,262797249,12/15/1995,104,en,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",8844,tt0113497,6.9,2413,19.386893
2,3,Waiting to Exhale,16000000,81452156,12/22/1995,127,en,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'iso_639_1': 'en', 'name': 'English'}]",31357,tt0114885,6.1,34,18.215526
3,4,Father of the Bride Part II,0,76578911,2/10/1995,106,en,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'id': 35, 'name': 'Comedy'}]","[{'iso_639_1': 'en', 'name': 'English'}]",11862,tt0113041,5.7,173,18.153832
4,5,Heat,60000000,187436818,12/15/1995,170,en,"[{'name': 'Regency Enterprises', 'id': 508}, {...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",949,tt0113277,7.7,1886,19.048952


In [56]:
def main_genre(row):
    return literal_eval(row.genres)[0]['name']


temp['main_genre'] = temp.apply(lambda row: main_genre(row), axis=1)
temp.head()

,Unnamed: 0,title,budget,revenue,release_date,runtime,original_language,production_companies,production_countries,genres,spoken_languages,id,imdb_id,vote_average,vote_count,log rev,main_genre
0,0,Toy Story,30000000,373554033,10/30/1995,81,en,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'iso_639_1': 'en', 'name': 'English'}]",862,tt0114709,7.7,5415,19.738573,Animation
1,1,Jumanji,65000000,262797249,12/15/1995,104,en,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",8844,tt0113497,6.9,2413,19.386893,Adventure
2,3,Waiting to Exhale,16000000,81452156,12/22/1995,127,en,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'iso_639_1': 'en', 'name': 'English'}]",31357,tt0114885,6.1,34,18.215526,Comedy
3,4,Father of the Bride Part II,0,76578911,2/10/1995,106,en,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'id': 35, 'name': 'Comedy'}]","[{'iso_639_1': 'en', 'name': 'English'}]",11862,tt0113041,5.7,173,18.153832,Comedy
4,5,Heat,60000000,187436818,12/15/1995,170,en,"[{'name': 'Regency Enterprises', 'id': 508}, {...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",949,tt0113277,7.7,1886,19.048952,Action


In [59]:
temp.columns

Index(['Unnamed: 0', 'title', 'budget', 'revenue', 'release_date', 'runtime',
       'original_language', 'production_companies', 'production_countries',
       'genres', 'spoken_languages', 'id', 'imdb_id', 'vote_average',
       'vote_count', 'log rev', 'main_genre'],
      dtype='object')

In [62]:
X=temp.iloc[ : , [2,3,4,5,6,7,8,10,13,14,]]
X

,budget,revenue,release_date,runtime,original_language,production_companies,production_countries,spoken_languages,vote_average,vote_count
0,30000000,373554033,10/30/1995,81,en,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",7.7,5415
1,65000000,262797249,12/15/1995,104,en,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",6.9,2413
2,16000000,81452156,12/22/1995,127,en,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",6.1,34
3,0,76578911,2/10/1995,106,en,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",5.7,173
4,60000000,187436818,12/15/1995,170,en,"[{'name': 'Regency Enterprises', 'id': 508}, {...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",7.7,1886
5,35000000,64350171,12/22/1995,106,en,"[{'name': 'Universal Pictures', 'id': 33}, {'n...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",5.5,174
6,58000000,352194034,11/16/1995,130,en,"[{'name': 'United Artists', 'id': 60}, {'name'...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",6.6,1194
7,62000000,107879496,11/17/1995,106,en,"[{'name': 'Columbia Pictures', 'id': 5}, {'nam...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",6.5,199
8,0,11348324,12/22/1995,78,en,"[{'name': 'Universal Pictures', 'id': 33}, {'n...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",7.1,423
9,44000000,13681765,12/22/1995,192,en,"[{'name': 'Hollywood Pictures', 'id': 915}, {'...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",7.1,72
